In [113]:
import pandas as pd
import os
import json
import re
import ast


### 1. Preprocessing

In [2]:
p1_files = os.listdir("part_1/")
data_p1 = pd.DataFrame()
for file in p1_files:
    if file.endswith(".csv"):
        data = pd.read_csv("part_1/" + file, low_memory=False)
        data_p1 = pd.concat([data_p1, data], axis=0)


In [3]:
p2_files = os.listdir("part_2/")
data_p2 = pd.DataFrame()
for file in p2_files:
    if file.endswith(".csv"):
        data = pd.read_csv("part_2/" + file, low_memory=False)
        data_p2 = pd.concat([data_p2, data], axis=0)


In [4]:
p3_files = os.listdir("part_3/")
data_p3 = pd.DataFrame()
for file in p3_files:
    if file.endswith(".csv"):
        data = pd.read_csv("part_3/" + file, low_memory=False)
        data_p3 = pd.concat([data_p3, data], axis=0)


In [5]:
p4_files = os.listdir("part_4/")
data_p4 = pd.DataFrame()
for file in p4_files:
    if file.endswith(".csv"):
        data = pd.read_csv("part_4/" + file, low_memory=False)
        data_p4 = pd.concat([data_p4, data], axis=0)


In [6]:
p5_files = os.listdir("part_5/")
data_p5 = pd.DataFrame()
for file in p5_files:
    if file.endswith(".csv"):
        data = pd.read_csv("part_5/" + file, low_memory=False)
        data_p5 = pd.concat([data_p5, data], axis=0)


In [7]:
print("Top 3 language in part 1:", data_p1.lang.value_counts()[0:3])
print("Top 3 language in part 2:", data_p2.lang.value_counts()[0:3])
print("Top 3 language in part 3:", data_p3.lang.value_counts()[0:3])
print("Top 3 language in part 4:", data_p4.lang.value_counts()[0:3])
print("Top 3 language in part 5:", data_p5.lang.value_counts()[0:3])


Top 3 language in part 1: lang
en     883111
qme     26206
es      20886
Name: count, dtype: int64
Top 3 language in part 2: lang
en     882757
qme     26912
es      17786
Name: count, dtype: int64
Top 3 language in part 3: lang
en     859551
qme     23708
es      14482
Name: count, dtype: int64
Top 3 language in part 4: lang
en     904990
qme     22734
es      17931
Name: count, dtype: int64
Top 3 language in part 5: lang
en     891663
qme     30762
es      13910
Name: count, dtype: int64


### For simplicity and the RQ, we only consider "en" and "es" posts. (es is also the second most popular languages in the dataset) 

In [8]:
data_p1 = data_p1[(data_p1['lang'] == 'en') | (data_p1['lang'] == 'es')]
data_p2 = data_p2[(data_p2['lang'] == 'en') | (data_p2['lang'] == 'es')]
data_p3 = data_p3[(data_p3['lang'] == 'en') | (data_p3['lang'] == 'es')]
data_p4 = data_p4[(data_p4['lang'] == 'en') | (data_p4['lang'] == 'es')]
data_p5 = data_p5[(data_p5['lang'] == 'en') | (data_p5['lang'] == 'es')]


In [9]:
data_p1 = data_p1[data_p1['text'].notna()]
data_p1 = data_p1[data_p1['text'].str.strip() != '']

data_p2 = data_p2[data_p2['text'].notna()]
data_p2 = data_p2[data_p2['text'].str.strip() != '']

data_p3 = data_p3[data_p3['text'].notna()]
data_p3 = data_p3[data_p3['text'].str.strip() != '']

data_p4 = data_p4[data_p4['text'].notna()]
data_p4 = data_p4[data_p4['text'].str.strip() != '']

data_p5 = data_p5[data_p5['text'].notna()]
data_p5 = data_p5[data_p5['text'].str.strip() != '']


In [44]:
data = pd.DataFrame()

In [45]:
data = pd.concat([data, data_p1])

In [46]:
data = pd.concat([data, data_p2])

In [47]:
data = pd.concat([data, data_p3])

In [48]:
data = pd.concat([data, data_p4])

In [49]:
data = pd.concat([data, data_p5])

In [146]:
data.shape

(4507066, 30)

In [50]:
data.reset_index(drop=True, inplace=True)


In [51]:
columns_to_keep = [
    'id', 'text', 'lang', 'epoch', 'hashtags', 'links', 
    'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 
    'conversationId', 'mentionedUsers', 'user'
]
data = data[columns_to_keep]


In [52]:
def user_to_dict(user):
    try:
        # User metadata interested in
        user_metadata = ['id_str', 'followersCount', 'friendsCount', 'statusesCount']
        
        metadata_dict = {}
        
        for metadata in user_metadata:
            pattern = rf"'{metadata}':\s*('[^']*'|\d+|True|False|None)"
            match = re.search(pattern, user)
            
            if match:
                metadata_dict[metadata] = match.group(1)  # Extract the matched value
            else:
                metadata_dict[metadata] = None
        
        return metadata_dict
    
    except Exception as e:
        print("Error occurred:", e)
        print("User data:", user)
        raise

In [53]:
data['user'] = data['user'].apply(user_to_dict)

In [60]:
data.user[0]['followersCount']

'19692'

In [62]:
# Flatten user metadata into separate columns
user_metadata = ['id_str', 'followersCount', 'friendsCount', 'statusesCount']
for meta in user_metadata:
    data[meta] = data['user'].apply(lambda x: x.get(meta))



In [65]:
data.drop(columns=['user'], inplace=True)

In [78]:
data.columns

Index(['id', 'text', 'lang', 'epoch', 'hashtags', 'links', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId',
       'mentionedUsers', 'id_str', 'followersCount', 'friendsCount',
       'statusesCount'],
      dtype='object')

In [144]:
# data.to_csv("twitter_data.csv",index=False)

In [ ]:
def convert_format(field):
    return ast.literal_eval(field)

# Convert string representation of lists into lists
data['hashtags'] = data['hashtags'].apply(convert_format)
data['links'] = data['links'].apply(convert_format)
data['mentionedUsers'] = data['mentionedUsers'].apply(convert_format)


### 2. Data exploration

In [94]:
en_data.columns

Index(['id', 'text', 'lang', 'epoch', 'hashtags', 'links', 'replyCount',
       'retweetCount', 'likeCount', 'quoteCount', 'conversationId',
       'mentionedUsers', 'id_str', 'followersCount', 'friendsCount',
       'statusesCount'],
      dtype='object')

In [148]:
en_data = data[data['lang'] == 'en']
es_data = data[data['lang'] == 'es']

In [159]:
print("Overall tweet counts:", data.shape[0])
print("English tweet counts:", en_data.shape[0])
print("Spanish tweet counts:", es_data.shape[0])

Overall tweet counts: 4507066
English tweet counts: 4422071
Spanish tweet counts: 84995


#### 2.1 Popular hashtags

In [149]:
from collections import Counter

en_tags = []
es_tags = []

for hashtags in en_data['hashtags']:
    if len(hashtags):
        en_tags.extend(tag['text'] for tag in hashtags if 'text' in tag)
        
for hashtags in es_data['hashtags']:
    if len(hashtags):
        es_tags.extend(tag['text'] for tag in hashtags if 'text' in tag)
        
en_tag_freq = Counter(en_tags)
es_tag_freq = Counter(es_tags)

In [157]:
total_en_tags = sum(en_tag_freq.values())  
total_es_tags = sum(es_tag_freq.values())  

en_tag_percent = Counter({tag: (count / total_en_tags) * 100 for tag, count in en_tag_freq.items()})
es_tag_percent = Counter({tag: (count / total_es_tags) * 100 for tag, count in es_tag_freq.items()})

print("\nTop 10 English Tags by Frequency %:")
for tag, percent in en_tag_percent.most_common(10):
    print(f"{tag}: {percent:.2f}%")

print("\nTop 10 Spanish Tags by Frequency %:")
for tag, percent in es_tag_percent.most_common(10):
    print(f"{tag}: {percent:.2f}%")


Top 10 English Tags by Frequency:
Trump2024: 4.78%
MAGA: 4.68%
Biden: 2.64%
Trump: 1.82%
BidenHarris2024: 1.37%
GOP: 0.86%
FoxNews: 0.84%
maga: 0.79%
VoteBlue2024: 0.67%
TRUMP2024: 0.65%

Top 10 Spanish Tags by Frequency:
Biden: 5.01%
EEUU: 2.60%
Mundo: 2.39%
Trump: 1.48%
JoeBiden: 1.30%
EstadosUnidos: 1.15%
Internacional: 1.14%
Trump2024: 1.08%
USA: 1.03%
Internacionales: 1.00%


#### 2.2 Popular links

In [163]:
en_links = []
es_links = []

for links in en_data['links']:
    if len(links):
        en_links.extend(link['expanded_url'] for link in links if 'expanded_url' in link)
        
for links in es_data['links']:
    if len(links):
        es_links.extend(link['expanded_url'] for link in links if 'expanded_url' in link)
        
en_link_freq = Counter(en_links)
es_link_freq = Counter(es_links)

In [164]:
total_en_links = sum(en_link_freq.values())  
total_es_links = sum(es_link_freq.values())  

en_link_percent = Counter({link: (count / total_en_links) * 100 for link, count in en_link_freq.items()})
es_link_percent = Counter({link: (count / total_es_links) * 100 for link, count in es_link_freq.items()})

print("\nTop 10 English Links by Frequency %:")
for link, percent in en_link_percent.most_common(10):
    print(f"{link}: {percent:.2f}%")

print("\nTop 10 Spanish Links by Frequency %:")
for link, percent in es_link_percent.most_common(10):
    print(f"{link}: {percent:.2f}%")


Top 10 English Links by Frequency %:
https://www.foxnews.com/politics/biden-dhs-docs-suggested-trump-supporters-military-religious-people-likely-violent-terror-threats: 0.10%
https://www.foxnews.com/politics/federal-judge-blocks-biden-title-ix-rule-4-states-abuse-power: 0.08%
https://opr.as/share: 0.08%
https://dailym.ai/ios: 0.08%
https://www.foxnews.com/media/pelosi-calls-trumps-family-republican-party-stage-intervention-trump-cult-thug: 0.07%
http://msn.com: 0.07%
https://www.foxnews.com/politics/house-committee-subpoenas-15-biden-cabinet-secretaries-hand-over-docs-voter-mobilization-scheme: 0.07%
http://Newsmax.com: 0.06%
https://www.foxnews.com/us/rachel-morin-murder-illegal-immigrant-from-el-salvador-charged-rape-killing-maryland-mom-5: 0.06%
https://www.foxnews.com/politics/president-biden-appears-freeze-white-house-juneteenth-event: 0.06%

Top 10 Spanish Links by Frequency %:
https://www.clarin.com/mundo/millonaria-batalla-donaciones-sacude-duelo-joe-biden-donald-trump_0_lTage

#### Check the amount of bilingual posters (case study)

In [79]:
df_en = data[data['lang'] == 'en']  
df_es = data[data['lang'] == 'es']  

users_en = set(df_en['id_str'])  
users_es = set(df_es['id_str'])

bilingual_posters = users_en.intersection(users_es)

bilingual_posters = list(bilingual_posters)



In [82]:
print(len(bilingual_posters), "users posted in English and Spanish")

6071 users posted in English and Spanish
